### Building on top of these concepts:

If you haven't covered langmem and basic openai-agents-sdk go through these notebooks first:

1. [Understand LangMem Core APIs](https://colab.research.google.com/drive/1YJNrnQRMgeNTigIuWOfykt-Z5L_DDmsa?usp=sharing)

2. [Use LangMem Memory Tools with OpenAI Agents SDK](https://colab.research.google.com/drive/1xgSUeJPIBKyjpM868PsvmCZaCof-s2vB?usp=sharing)

3. [Use Persistence Storage](https://colab.research.google.com/drive/1gA9r_FkbHFCWlgd52qz1oTW736gD0kmN?usp=sharing)

4. [Baseline Email Assistant - Prep before Memory](https://colab.research.google.com/drive/1AgedinzRuoow3f2cvR0wRlATk8AgE0x0?usp=sharing)

5. [OpenAI Agents SDK Email Assistant with Semantic](https://colab.research.google.com/drive/1L4T8eZIHzYD1OQtC8hubZMeFy-eRmxV2?usp=sharing)

# **Episodic Memory**

While semantic memory builds knowledge ("what"), episodic memory captures expertise ("how").

### **When to Use Episodic Memory**?
Episodic memory drives adaptive learning. While semantic memory builds a knowledge base of facts ("Python is a programming language"), episodic memory captures the expertise of how to apply that knowledge effectively ("explaining Python using snake analogies confused users, but comparing it to recipe steps worked well").

This experience replay helps agents: - Adapt teaching style based on what worked - Learn from successful problem-solving approaches - Build a library of proven techniques - Understand not just what to do, but why it works



## Install Packages

In [1]:
!pip install -Uq openai-agents langmem langchain-google-genai langmem_adapter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")

# 1. **Episodic Memories Extraction**

In [4]:
from langmem import create_memory_manager
from pydantic import BaseModel, Field


class Episode(BaseModel):
    """Write the episode from the perspective of the agent within it. Use the benefit of hindsight to record the memory, saving the agent's key internal thought process so it can learn over time."""

    observation: str = Field(..., description="The context and setup - what happened")
    thoughts: str = Field(
        ...,
        description="Internal reasoning process and observations of the agent in the episode that let it arrive"
        ' at the correct action and result. "I ..."',
    )
    action: str = Field(
        ...,
        description="What was done, how, and in what format. (Include whatever is salient to the success of the action). I ..",
    )
    result: str = Field(
        ...,
        description="Outcome and retrospective. What did you do well? What could you do better next time? I ...",
    )


#  The Episode schema becomes part of the memory manager's prompt,
# helping it extract complete reasoning chains that guide future responses
manager = create_memory_manager(
    "google_genai:gemini-2.0-flash",
    schemas=[Episode],
    instructions="Extract examples of successful explanations, capturing the full chain of reasoning. Be concise in your explanations and precise in the logic of your reasoning.",
    enable_inserts=True,
)

In [5]:
conversation = [
    {
        "role": "user",
        "content": "What's a binary tree? I work with family trees if that helps",
    },
    {
        "role": "assistant",
        "content": "A binary tree is like a family tree, but each parent has at most 2 children. Here's a simple example:\n   Bob\n  /  \\\nAmy  Carl\n\nJust like in family trees, we call Bob the 'parent' and Amy and Carl the 'children'.",
    },
    {
        "role": "user",
        "content": "Oh that makes sense! So in a binary search tree, would it be like organizing a family by age?",
    },
]

episodes = manager.invoke({"messages": conversation})

print(episodes)

[ExtractedMemory(id='38ce4640-3838-4ce1-9e6d-51e0d164f976', content=Episode(observation='User asked what a binary tree is and mentioned their familiarity with family trees.', thoughts="I used the user's existing knowledge of family trees to explain the concept of a binary tree, which seemed to resonate with them. I need to remember this analogy for future explanations.", action='Explained binary tree using a family tree analogy, showing a simple example with parent and children.', result='The user understood the explanation.'))]


In [6]:
for episode in episodes:
  print(episode.content, "\n\n")

observation='User asked what a binary tree is and mentioned their familiarity with family trees.' thoughts="I used the user's existing knowledge of family trees to explain the concept of a binary tree, which seemed to resonate with them. I need to remember this analogy for future explanations." action='Explained binary tree using a family tree analogy, showing a simple example with parent and children.' result='The user understood the explanation.' 




Now you can store and use it as part of your agent context i.e: instructions

# 2.  **OpenAI Agents SDK Email Assistant with Semantic + Episodic Memory**

We previously built an email assistant that:
- Classifies incoming messages (respond, ignore, notify)
- Drafts responses
- Schedules meetings
- Uses memory to remember details from previous emails

Now, we'll add human-in-the-loop following the triage step to better refine the assistant's ability to classify emails.

## OpenAI Agents SDK Model Config

In [7]:
from pydantic import BaseModel, Field
from agents import (
    Agent,
    Runner,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)
from typing import Dict, Any
from typing_extensions import TypedDict, Literal, Annotated


In [8]:
#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=os.environ["GOOGLE_API_KEY"],
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)


## **Setup a Profile, Prompt Instructions and Example Email**

In [9]:
profile = {
    "name": "Junaid",
    "full_name": "Muhammad Junaid Shaukat",
    "user_profile_background": "AI Engineer building personal AI Agents WorkForce.",
}

In [10]:
prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage Junaid tasks efficiently."
}

## Few Shot-examples - Label & Store

### 1. Define Memory Storage Layer

In [11]:
import asyncio
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langgraph.store.memory import InMemoryStore
from contextlib import asynccontextmanager

store = InMemoryStore(
      index={
          "dims": 768,
          "embed": GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
      }
    )

@asynccontextmanager
async def get_store():
  yield store


### 2. Label and Store Few Shot Examples

In [12]:
# Example incoming email
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>",
    "subject": "Quick question about AI documentation",
    "body": """
Hi Junaid,

I was reviewing the AI Agents documentation for the new agentic authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /agent/auth/refresh
- /agent/auth/validate

Thanks!
Alice""",
}

In [13]:
data = {
    "email": email,
    # This is to start changing the behavior of the agent
    "label": "respond"
}

#### store example into memory store using 'examples' to indicate episodic memory

In [14]:
import uuid
store.put(
    ("email_assistant", "Junaid", "examples"),
    str(uuid.uuid4()),
    data
)

### Store a Second Example

In [15]:
data = {
    "email": {
        "author": "Sarah Chen <sarah.chen@company.com>",
    "to": "Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>",
        "subject": "Update: Agentic API Changes Deployed to Staging",
        "email_thread": """Hi Junaid,

    Just wanted to let you know that I've deployed the new authentication endpoints we discussed to the staging environment. Key changes include:

    - Implemented JWT refresh token rotation
    - Added rate limiting for login attempts
    - Updated API documentation with new endpoints

    All tests are passing and the changes are ready for review. You can test it out at staging-api.company.com/auth/*

    No immediate action needed from your side - just keeping you in the loop since this affects the systems you're working on.

    Best regards,
    Sarah
    """,
    },
    "label": "ignore"
}

In [16]:
store.put(
    ("email_assistant", "Junaid", "examples"),
    str(uuid.uuid4()),
    data
)

In [17]:
store.search(("email_assistant", "Junaid", "examples"))

[Item(namespace=['email_assistant', 'Junaid', 'examples'], key='91bd9944-1e73-429f-8ed7-aa0d502478ac', value={'email': {'from': 'Alice Smith <alice.smith@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 'subject': 'Quick question about AI documentation', 'body': "\nHi Junaid,\n\nI was reviewing the AI Agents documentation for the new agentic authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /agent/auth/refresh\n- /agent/auth/validate\n\nThanks!\nAlice"}, 'label': 'respond'}, created_at='2025-03-31T13:02:11.573718+00:00', updated_at='2025-03-31T13:02:11.573725+00:00', score=None),
 Item(namespace=['email_assistant', 'Junaid', 'examples'], key='4f809d93-ac5b-4ea3-a48b-fd0b669c11dc', value={'email': {'author': 'Sarah Chen <sarah.chen@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 's

### Simulate searching and returning examples

In [18]:
# Template for formating an example to put in prompt
template = """Email Subject: {subject}
Email From: {from_email}
Email To: {to_email}
Email Content:
```
{content}
```
> Triage Result: {result}"""

# Format list of few shots
def format_few_shot_examples(examples):
    print(examples)
    strs = ["Here are some previous examples:"]
    for eg in examples:
        email_data = eg.value.get("email", {})
        subject = email_data.get("subject", "No Subject")
        to_email = email_data.get("to", "No Recipient")
        from_email = email_data.get("author") or email_data.get("from", "No Sender")
        # Use 'email_thread' if available; otherwise fall back to 'body'
        content = email_data.get("email_thread") or email_data.get("body", "")
        # Truncate the content to a maximum of 400 characters
        content = content[:400]
        result = eg.value.get("label", "No Label")

        strs.append(
            template.format(
                subject=subject,
                to_email=to_email,
                from_email=from_email,
                content=content,
                result=result,
            )
        )
    return "\n\n------------\n\n".join(strs)

Whenever we get an email we will use it to search in store to get relevant emails

In [19]:
email_data = {
        "author": "Sarah Chen <sarah.chen@company.com>",
        "to": "Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>",
        "subject": "Update: Backend API Changes Deployed to Staging",
        "email_thread": """Hi Junaid,

    Wanted to let you know that I've deployed the new authentication endpoints we discussed to the staging environment. Key changes include:

    - Implemented JWT refresh token rotation
    - Added rate limiting for login attempts
    - Updated API documentation with new endpoints

    All tests are passing and the changes are ready for review. You can test it out at staging-api.company.com/auth/*

    No immediate action needed from your side - just keeping you in the loop since this affects the systems you're working on.

    Best regards,
    Sarah
    """,
    }


In [20]:
results = store.search(
    ("email_assistant", "Junaid", "examples"),
    query=str({"email": email_data}),
    limit=1)

In [21]:
print(format_few_shot_examples(results))

[Item(namespace=['email_assistant', 'Junaid', 'examples'], key='4f809d93-ac5b-4ea3-a48b-fd0b669c11dc', value={'email': {'author': 'Sarah Chen <sarah.chen@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 'subject': 'Update: Agentic API Changes Deployed to Staging', 'email_thread': "Hi Junaid,\n\n    Just wanted to let you know that I've deployed the new authentication endpoints we discussed to the staging environment. Key changes include:\n\n    - Implemented JWT refresh token rotation\n    - Added rate limiting for login attempts\n    - Updated API documentation with new endpoints\n\n    All tests are passing and the changes are ready for review. You can test it out at staging-api.company.com/auth/*\n\n    No immediate action needed from your side - just keeping you in the loop since this affects the systems you're working on.\n\n    Best regards,\n    Sarah\n    "}, 'label': 'ignore'}, created_at='2025-03-31T13:02:23.690867+00:00', updated_at='2025-03-31T13

## Code

In [22]:
class Email(BaseModel):
    from_: str = Field(alias="from")
    to: str
    subject: str
    body: str

email_model = Email(**email)
print(email_model.model_dump_json(by_alias=True))

{"from":"Alice Smith <alice.smith@company.com>","to":"Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>","subject":"Quick question about AI documentation","body":"\nHi Junaid,\n\nI was reviewing the AI Agents documentation for the new agentic authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /agent/auth/refresh\n- /agent/auth/validate\n\nThanks!\nAlice"}


In [23]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

## **Prompt Templates**

In [24]:
# Triage system prompt template
triage_system_prompt_template = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Background >
{user_profile_background}.
</ Background >

< Instructions >

{name} gets lots of emails. Your job is to categorize each email into one of three categories:

1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but doesn't require a response
3. RESPOND - Emails that need a direct response from {name}

Classify the below email into one of these categories.

</ Instructions >

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email response. For these, you should notify {name} (using the `notify` response). Examples of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >
{examples}
</ Few shot examples >
"""





# Triage User Prompt Template
triage_user_prompt_template = """
Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}
"""





## **A function to create a prompt using f-string**

In [25]:
def create_prompt(template: str, variables: Dict[str, any]) -> str:
    """Creates a prompt using an f-string and a dictionary of variables."""
    try:
        return template.format(**variables)
    except KeyError as e:
        return f"Error: Missing variable '{e.args[0]}' in the provided dictionary."

## **Response Agent, Define Tools**

In [26]:
from agents import function_tool

In [27]:
@function_tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'"

In [28]:
@function_tool
def schedule_meeting(
    attendees: list[str],
    subject: str,
    duration_minutes: int,
    preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"



In [29]:
@function_tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

### **LangMem Memory Management Tools**

In [30]:
class UserInfo(BaseModel):
  username: str

namespace_template=("email_assistant", "{username}", "collection")

In [31]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langmem_adapter import LangMemOpenAIAgentToolAdapter

# Initialize the manage memory tool dynamically:
manage_adapter = LangMemOpenAIAgentToolAdapter(
    lambda store, namespace=None: create_manage_memory_tool(namespace=namespace, store=store),
    store_provider=get_store,
    namespace_template=namespace_template
)
manage_memory_tool = manage_adapter.as_tool()

# Initialize the search memory tool dynamically:
search_adapter = LangMemOpenAIAgentToolAdapter(
    lambda store, namespace=None: create_search_memory_tool(namespace=namespace, store=store),
    store_provider=get_store,
    namespace_template=namespace_template
)
search_memory_tool = search_adapter.as_tool()


In [32]:
store.search(("email_assistant", "Junaid", "examples"))

[Item(namespace=['email_assistant', 'Junaid', 'examples'], key='91bd9944-1e73-429f-8ed7-aa0d502478ac', value={'email': {'from': 'Alice Smith <alice.smith@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 'subject': 'Quick question about AI documentation', 'body': "\nHi Junaid,\n\nI was reviewing the AI Agents documentation for the new agentic authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /agent/auth/refresh\n- /agent/auth/validate\n\nThanks!\nAlice"}, 'label': 'respond'}, created_at='2025-03-31T13:02:11.573718+00:00', updated_at='2025-03-31T13:02:11.573725+00:00', score=None),
 Item(namespace=['email_assistant', 'Junaid', 'examples'], key='4f809d93-ac5b-4ea3-a48b-fd0b669c11dc', value={'email': {'author': 'Sarah Chen <sarah.chen@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 's

## **Response Agent, Define Prompt**

In [33]:
response_prompt_template = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

In [34]:
response_system_prompt = create_prompt(response_prompt_template, {
    "full_name": profile["full_name"],
    "name":profile["name"],
    "instructions": prompt_instructions["agent_instructions"] + "Always save my email interactions in memory store for later discussions.",
  }
)
print(response_system_prompt)


< Role >
You are Muhammad Junaid Shaukat's executive assistant. You are a top-notch executive assistant who cares about Junaid performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage Junaid's communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
</ Tools >

< Instructions >
Use these tools when appropriate to help manage Junaid tasks efficiently.Always save my email interactions in memory store for later discussions.
</ Instructions >



In [35]:
tools=[write_email, schedule_meeting, check_calendar_availability, manage_memory_tool, search_memory_tool]

In [36]:
response_agent = Agent[UserInfo](
    name="Response agent",
    instructions=response_system_prompt,
    tools=tools
    )


### Just Testing

In [37]:
response_result = await Runner.run(response_agent,
                                   "Muhammad is my friend",
                                   run_config = config,
                                   context=UserInfo(username=profile["name"])
                                   )
print(response_result.final_output)


Okay, I've saved that you are friends with Muhammad. How can I help you today, Junaid?



In [38]:
response_result = await Runner.run(response_agent,
                                   "Do I know Muhammad?",
                                   run_config = config,
                                   context=UserInfo(username=profile["name"])
                                   )
print(response_result.final_output)


Yes, you are friends with Muhammad.



In [39]:
async with get_store() as store:
  namespace=("email_assistant", profile["name"], "collection")
  res = await store.asearch(namespace)
  print(res)

[Item(namespace=['email_assistant', 'Junaid', 'collection'], key='769822d4-354a-4681-8441-22df65666bb2', value={'content': 'The user is friends with Muhammad.'}, created_at='2025-03-31T13:03:18.623944+00:00', updated_at='2025-03-31T13:03:18.623953+00:00', score=None)]


## **Create Triage Agent Flow in Python**

In [40]:
async def triage_router(email: Email, username: str):

    namespace = (
        "email_assistant",
        username,
        "examples"
    )

    examples = store.search(
        namespace,
        query=str({"email": email.model_dump()}),
    )
    examples=format_few_shot_examples(examples)

    system_prompt = create_prompt(triage_system_prompt_template, {
        "full_name": profile["full_name"],
        "name":profile["name"],
        "user_profile_background": profile["user_profile_background"],
        "triage_no" : prompt_instructions["triage_rules"]["ignore"],
        "triage_notify": prompt_instructions["triage_rules"]["notify"],
        "triage_email" : prompt_instructions["triage_rules"]["respond"],
        "examples": examples,
      }
    )

    user_prompt = create_prompt(triage_user_prompt_template, {
      "author": email.from_,
      "to": email.to,
      "subject": email.subject,
      "email_thread" : email.body
    })

    triage_agent = Agent(
        name="Triage Agent",
        instructions=system_prompt,
        output_type=Router
    )

    triage_result = await Runner.run(
        triage_agent,
        user_prompt,
        run_config = config,
        context=UserInfo(username=username)
        )

    print(triage_result.final_output)
    print("Triage History: ", triage_result.to_input_list())

    if triage_result.final_output.classification == "respond":
          print("📧 Classification: RESPOND - This email requires a response")
          response_result = await Runner.run(
              response_agent,
              f"Respond to the email {email.model_dump_json(by_alias=True)}",
              run_config = config,
              context=UserInfo(username=username)
              )
          print(response_result.final_output)
          print("Response History", response_result.to_input_list())
    elif triage_result.final_output.classification == "ignore":
        print("🚫 Classification: IGNORE - This email can be safely ignored")
    elif triage_result.final_output.classification == "notify":
        # If real life, this would do something else
        print("🔔 Classification: NOTIFY - This email contains important information")
    else:
        raise ValueError(f"Invalid classification: {triage_result.final_output.classification}")


## **Now Test the Triage and Response Agents Working Together**

In [41]:
email_input = {
    "from": "Tom Jones <tome.jones@bar.com>",
    "to": "Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>",
    "subject": "Purchase API documentation",
    "body": """Hi John - I want to buy documentation?""",
}

typed_email = Email(**email_input)
print(typed_email.model_dump_json(by_alias=True))

{"from":"Tom Jones <tome.jones@bar.com>","to":"Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>","subject":"Purchase API documentation","body":"Hi John - I want to buy documentation?"}


In [42]:
await triage_router(typed_email, "Junaid")

[Item(namespace=['email_assistant', 'Junaid', 'examples'], key='91bd9944-1e73-429f-8ed7-aa0d502478ac', value={'email': {'from': 'Alice Smith <alice.smith@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 'subject': 'Quick question about AI documentation', 'body': "\nHi Junaid,\n\nI was reviewing the AI Agents documentation for the new agentic authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /agent/auth/refresh\n- /agent/auth/validate\n\nThanks!\nAlice"}, 'label': 'respond'}, created_at='2025-03-31T13:02:11.573718+00:00', updated_at='2025-03-31T13:02:11.573725+00:00', score=0.7109069218571045), Item(namespace=['email_assistant', 'Junaid', 'examples'], key='4f809d93-ac5b-4ea3-a48b-fd0b669c11dc', value={'email': {'author': 'Sarah Chen <sarah.chen@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gm

In [43]:
email_input2 = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>",
    "subject": "Urgent Help for API documentation",
    "body": """Hi Junaid,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

typed_email2 = Email(**email_input2)

In [44]:
await triage_router(typed_email2, "Junaid")

[Item(namespace=['email_assistant', 'Junaid', 'examples'], key='91bd9944-1e73-429f-8ed7-aa0d502478ac', value={'email': {'from': 'Alice Smith <alice.smith@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 'subject': 'Quick question about AI documentation', 'body': "\nHi Junaid,\n\nI was reviewing the AI Agents documentation for the new agentic authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /agent/auth/refresh\n- /agent/auth/validate\n\nThanks!\nAlice"}, 'label': 'respond'}, created_at='2025-03-31T13:02:11.573718+00:00', updated_at='2025-03-31T13:02:11.573725+00:00', score=0.8604749487463565), Item(namespace=['email_assistant', 'Junaid', 'examples'], key='4f809d93-ac5b-4ea3-a48b-fd0b669c11dc', value={'email': {'author': 'Sarah Chen <sarah.chen@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gm

#### Update store to ignore emails like this

In [45]:
data = {
    "email": {
    "from": "Tom Jones <tome.jones@bar.com>",
    "to": "Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>",
    "subject": "Quick question about API documentation",
    "body": """Hi Junaid - want to buy documentation?""",
},
    "label": "ignore"
}

In [46]:
store.put(
    ("email_assistant", "Junaid", "examples"),
    str(uuid.uuid4()),
    data
)

#### Try it, it should ignore

In [47]:
email_input = {
    "from": "Tom Jones <tome.jones@bar.com>",
    "to": "Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>",
    "subject": "Quick question about API documentation",
    "body": """Hi Junaid - want to buy documentation?""",
}

In [48]:
await triage_router(Email(**email_input), "Junaid")

[Item(namespace=['email_assistant', 'Junaid', 'examples'], key='c2d59ad4-21f4-4aca-b76c-df3c187b831c', value={'email': {'from': 'Tom Jones <tome.jones@bar.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 'subject': 'Quick question about API documentation', 'body': 'Hi Junaid - want to buy documentation?'}, 'label': 'ignore'}, created_at='2025-03-31T13:04:45.550475+00:00', updated_at='2025-03-31T13:04:45.550483+00:00', score=0.8548454868732505), Item(namespace=['email_assistant', 'Junaid', 'examples'], key='91bd9944-1e73-429f-8ed7-aa0d502478ac', value={'email': {'from': 'Alice Smith <alice.smith@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 'subject': 'Quick question about AI documentation', 'body': "\nHi Junaid,\n\nI was reviewing the AI Agents documentation for the new agentic authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the 

#### Modifying text, may continue to ignore

In [49]:
email_input = {
    "from": "Tom Jones <tome.jones@gmail.com>",
    "to": "Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>",
    "subject": "API documentation Inquiry",
    "body": """Hi Junaid - want to ask some questions about API. When can we connect and I wanna make an offer to buy documentation?""",
}

In [50]:
await triage_router(Email(**email_input), "Junaid")

[Item(namespace=['email_assistant', 'Junaid', 'examples'], key='c2d59ad4-21f4-4aca-b76c-df3c187b831c', value={'email': {'from': 'Tom Jones <tome.jones@bar.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 'subject': 'Quick question about API documentation', 'body': 'Hi Junaid - want to buy documentation?'}, 'label': 'ignore'}, created_at='2025-03-31T13:04:45.550475+00:00', updated_at='2025-03-31T13:04:45.550483+00:00', score=0.8390072792146372), Item(namespace=['email_assistant', 'Junaid', 'examples'], key='91bd9944-1e73-429f-8ed7-aa0d502478ac', value={'email': {'from': 'Alice Smith <alice.smith@company.com>', 'to': 'Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>', 'subject': 'Quick question about AI documentation', 'body': "\nHi Junaid,\n\nI was reviewing the AI Agents documentation for the new agentic authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the 

In [51]:
namespace=("email_assistant", "Junaid", "collection")
res = store.search(namespace)
print(res)

[Item(namespace=['email_assistant', 'Junaid', 'collection'], key='769822d4-354a-4681-8441-22df65666bb2', value={'content': 'The user is friends with Muhammad.'}, created_at='2025-03-31T13:03:18.623944+00:00', updated_at='2025-03-31T13:03:18.623953+00:00', score=None), Item(namespace=['email_assistant', 'Junaid', 'collection'], key='86bf9288-499a-4f94-b79f-7290b50a862b', value={'content': 'Received email from Tom Jones about purchasing API documentation. Sent a reply requesting more details about the specific API documentation he needs.'}, created_at='2025-03-31T13:04:12.445749+00:00', updated_at='2025-03-31T13:04:12.445756+00:00', score=None), Item(namespace=['email_assistant', 'Junaid', 'collection'], key='7ccc1405-5339-4a71-9f27-1c61a70cde99', value={'content': 'Responded to Alice Smith regarding missing endpoints in API documentation and will follow up with the development team.'}, created_at='2025-03-31T13:04:34.515381+00:00', updated_at='2025-03-31T13:04:34.515389+00:00', score=No

#### Try with a different user id

In [52]:
await triage_router(Email(**email_input), "ahmad")

[]
reasoning='This is a direct inquiry about the API documentation and a potential offer to buy it, so Junaid needs to respond.' classification='respond'
Triage History:  [{'content': '\nPlease determine how to handle the below email thread:\n\nFrom: Tom Jones <tome.jones@gmail.com>\nTo: Muhammad Junaid Shaukat <mr.junaidshaukat@gmail.com>\nSubject: API documentation Inquiry\nHi Junaid - want to ask some questions about API. When can we connect and I wanna make an offer to buy documentation?\n', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': '{\n  "reasoning": "This is a direct inquiry about the API documentation and a potential offer to buy it, so Junaid needs to respond.",\n  "classification": "respond"\n}', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}]
📧 Classification: RESPOND - This email requires a response
Okay, I will respond to Tom Jones' email. What day would you like to schedule a meeting with him to 

In [53]:
namespace=("email_assistant", "ahmad", "collection")
res = store.search(namespace)
print(res)

[]
